# Tidy data in Pandas

### Handy hints 

Some Jupyter shortcut keys:

- Esc : get into Command mode (leaves Edit mode)
- Enter : edit a cell (puts you in Edit mode)
- h : see help (see all commands)
- Shift+Enter or Ctrl+Enter : run the code in the cell
- a / b : add new cell above/below
- m : turn current cell into a Markdown cell
- y : turn current cell into a Code cell
- 1-6 : turn current cell into a heading (a type of Markdown cell)

You can get help on Python objects and functions with `help()` or the `?` operator.

## Setup 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# This causes Jupyter to display any matplotlib plots directly in the notebook
# It also works for pandas and seaborn, since they use matplotlib to render plots
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# pyplot (as plt) is the module we'll primarily use to instantiate matplotlib plot objects
from matplotlib import pyplot as plt
import seaborn as sns

## Pandas 

You can find the Pandas website and documentation at https://pandas.pydata.org/

Pandas is a popular Python library for handling tabular data. It provides much of the same functionality for Python as do data frames in the R language. 

The fundamental data types in Pandas are a Series, representing a 1D array of data:

In [ ]:
s = pd.Series(["Diego","Jessica","Farah"])
s

and a DataFrame, representing a 2D table of data: 

In [ ]:
df = pd.DataFrame({'Name': ["Diego","Jessica","Farah"],
                   'Age': [34, 27, 50]})
df

We can think of a DataFrame like a spreadsheet or a table in a database; every column represents a variable in the dataset. Each column of a DataFrame has a particular type (ints, floats, datetimes, strings etc) and each column can be treated as a Pandas Series.

Above, we constructed data manually using lists and dicts. For the rest of this workshop, we will work with real data. DataFrames are the natural type to use when reading in tabular data from, for instance, CSV files or Excel files.

Here's example to read in some small datasets which we can use for demo purposes:

In [ ]:
iris = pd.read_csv('iris.csv')
cars = pd.read_csv('mtcars.csv')

## Data 

The data we'll use for this workshop is from the [Long Term Evolution Experiment (LTEE)](http://myxo.css.msu.edu/ecoli/). This experiment has been running for over 30 years and over 50,000 E. coli generations, and is still ongoing. Twelve separate populations of E. coli have been propagated for the life of the experiment. Every 500 generations, each population is cloned and stored, allowing researchers to study evolutionary behaviour over the long term, and to "rewind and replay" alternate evolutionary trajectories by propagating from an earlier clone. 

Several interesting events have occurred during the experiment. Some populations have spontaneously developed hypermutator phenotypes. In addition, around generation 31,000 one population, Ara-3, spontaneously developed a rare and novel Cit+ mutation, giving it the ability to metabolise citrate in the substrate.

There have been many publications from this experiment. A handful:

- [Blount et al 2008: Historical contingency and the evolution of a key innovation in an experimental population of Escherichia coli](https://www.pnas.org/content/105/23/7899) - on the spontaneous development of citrate metabolisation and on potentiating mutations
- [Tenaillon et al 2016: Tempo and mode of genome evolution in a 50,000-generation experiment](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4988878) - various investigations by sequencing and variant-calling over 50,000 generations of clones, including discussion of hypermutator phenotypes and genetic drift vs natural selection.

Sequence data from clones is available, but for this workshop we'll just be using some published tabular data.

A version of this dataset is also used by the [Data Carpentry lessons on Genomics](https://datacarpentry.org/genomics-workshop/).

In the repository you'll find the files:

- ltee_sampleruns.csv : sample and sequencing run metadata for the E. coli clones
- ltee_mutations.csv : analysis output from variant calling on the E. coli clones
- ltee_relative_fitness.tsv : relative fitness values for each population and generation up to generation 10,000
- ltee_cell_size.tsv : cell sizes for each population and generation up to 10,000

Let's read in the sample and run metadata. Pandas has functions for reading in many data types. Try looking at the documentation for `read_csv()` by running `help(pd.read_csv)` or `pd.read_csv?`.

In [ ]:
sampleruns = pd.read_csv('ltee_sampleruns.csv')

In [ ]:
# The number of rows and columns
sampleruns.shape

In [ ]:
# The first few rows
sampleruns.head()

In [ ]:
# Column data types
sampleruns.dtypes

In [ ]:
# Column names
sampleruns.columns

In [ ]:
# Index (i.e. row names)
# In this case we didn't provide an index and rows have simply been numbered for us by Pandas
sampleruns.index

#### Exercise

1. Use `pd.read_csv()` to read the file `ltee_mutations.csv` into a variable called `mutations`.
2. Check the column headings and the number of rows in this dataset, and have a look at the first few rows. Compare the size of the dataset and the variables to `sampleruns`.

In [ ]:
mutations = pd.read_csv('ltee_mutations.csv')

In [ ]:
mutations.head()

## Indexing and slicing 

We can extract a column from the DataFrame by indexing with square brackets, e.g. 

In [ ]:
# Extracting a column
sampleruns['Strain ID']

Firstly, let's set our index (row names) to something more meaningful to make it easier to see what's going on. The Strain ID uniquely identifies each sample, so it is probably a sensible index. We can use `sampleruns.set_index()`, or we can assign to the index directly:

In [ ]:
sampleruns.index = sampleruns['Strain ID']
sampleruns.head()

The two most important ways to extract data from a DataFrame are `loc` and `iloc`. `loc` uses the index and the column names; `iloc` uses the row and column numbers, counting from zero. 

In [ ]:
sampleruns.loc['REL768B', 'Accession']

In [ ]:
# Row 0, column 9
sampleruns.iloc[0,9]

In [ ]:
# Rows 1-3, column 9
sampleruns.iloc[1:4, 9]

In [ ]:
# All columns
sampleruns.loc['REL768B', :]

In [ ]:
# Lists let us specify any set of rows and columns, in any order
sampleruns.loc[['REL768A','REL958A'], ['Read Type', 'Read Length']]

In [ ]:
# We can also use True/False values to perform boolean indexing. 
# Pandas will return the rows/columns matching the True values we pass in.
# This will be useful later for filtering data
iris.loc[0:5, [True, False, True, True, False]]

#### Exercise

1. Set the index of `mutations` to be the same as the "Strain ID" column.
2. Extract the population, generation, and number of total mutations for strain REL11345

In [ ]:
mutations.index = mutations['Strain ID']

In [ ]:
mutations.loc['REL11345', ['Population','Generation', 'Total Mutations']]

## Manipulating Series

A single column of a DataFrame is a Series object. Series have a data type:

In [ ]:
sampleruns['Sequencing Depth'].dtype

Like a DataFrame, a Series has an index. In this case we got our Series from a column of a DataFrame, so it will have the same index:

In [ ]:
sampleruns['Sequencing Depth'].index

There are several convenience functions defined on Series. For instance, we can find the average sequencing depth:

In [ ]:
sampleruns['Sequencing Depth'].mean()

Similarly for numeric variables we have, for instance, `.min()` and `.max()`, `.median()`, `std()`, and `sum()`.

`.describe()` is a convenience function for getting several summary statistics at once:

In [ ]:
sampleruns['Sequencing Depth'].describe()

For non-numeric variable types such as strings and categoricals, we may want to look at the unique values.

In [ ]:
sampleruns['Read Type'].unique()

In [ ]:
sampleruns['Read Type'].value_counts()

Pandas makes use of numpy vectorisation, meaning we can do operations on Series with simple syntax, and it will be efficient to run.

In [ ]:
# TODO: use mutations unless exercise
sampleruns['Sequencing Depth'].head()

In [ ]:
sampleruns['Sequencing Depth'].head() + 5

You might have noticed that the `Analysis Notes` column contains a lot of NaN's. This means "not a number" and represents a missing value - i.e. these cells are empty. 

In [ ]:
sampleruns.head()

We can check which values are missing with `.isnull()`. This converts every value in the DataFrame (or Series) into a boolean True/False value.

In [ ]:
sampleruns.isnull().head()

Adding up booleans will treat `True` as `1` and `False` as `0`. A common approach is to use `sum()` to count how many `True` values there are. So we can count missing values like this:

In [ ]:
sampleruns.isnull().sum()

`sampleruns` had 264 rows, so it looks like there are a few non-empty note cells. We could count this explicitly by taking the logical `not` of our True/False values, i.e. adding up cells where `isnull()` is `False`. For manipulating array-like data, we can't use the `not`, `and` and `or` boolean operators. Instead we need to use the bitwise operators `~`, `&`, and `|`.

In [ ]:
(~sampleruns.isnull()).sum()

Notice that Pandas summed each column. We can use `sum(axis=1)` to override this default and sum each row instead.

## Sorting and filtering 

We can sort on a field, or list of fields, with `.sort_values()`:

In [ ]:
# Get a random example subset
subset = sampleruns.sample(15)
subset

In [ ]:
subset.sort_values('Generation')

In [ ]:
subset.sort_values(['Population','Generation'])

We can filter datasets using boolean indexing. This means that if we use a logical expression produce a boolean Series with a logical expression:

In [ ]:
subset['Population'] == 'Ara+5'

we can then select out only the rows (or sometimes columns) where that logical expression is True:

In [ ]:
subset.loc[subset['Population'] == 'Ara+5', :]

#### Exercise 

1. Filter the `sampleinfo` dataset to extract only rows which contain Analysis Notes, i.e. those where this field is not empty.
2. Filter the `mutations` dataset to extract only samples with more than 1500 total mutations. 
3. Sort the resulting data from (2) by the number of "Small Indels". Have a look at the resulting Population and Generation columns.


In [ ]:
mutations[mutations['Total Mutations'] > 1500].sort_values('Small Indels')

## Interlude: tidy data

[Link to "untidy data" discussion spreadsheet](https://docs.google.com/spreadsheets/d/1P94oEzgxNzlpvYiento53tZxJwaHYi8gpcdpDouu2jw/edit?usp=sharing)

(Don't look too far into this spreadsheet before we get up to it, as it includes solutions which are a spoiler for the exercise.)

## Reshaping data 

We can reshape data using particular columns, with `melt` and `pivot` or `pivot_table`. We'll have a look at this below.

We can also reshape data using the column names and index, with `stack` and `unstack`. This requires MultiIndexes, which we won't go into today.

Here are two tiny "wide" datasets based on our "untidy" housing data:

In [ ]:
sales_wide1 = pd.read_csv('housing-data-wide1.csv')
sales_wide1

In [ ]:
sales_wide2 = pd.read_csv('housing-data-wide2.csv', parse_dates=['date1','date2'])
sales_wide2

How can we reshape these into tidy form? 

The Pandas `melt` function will do this. For instance:

In [ ]:
sales_wide1.melt()

This has put every variable (i.e. every column) into the new `variable` column. This probably isn't what we want. It's only the price columns that are "wide", the other variables were fine.

In [ ]:
# Retain columns property_id and bedrooms
sales_wide1.melt(id_vars=['property_id','bedrooms'])

This is getting close to what we want. The `variable` column contains the original column names and tells us whether the price we're looking at was from the first or second sale (this may or may not be information we care about). The `value` column contains values in the melted columns, i.e. the actual price. 

Now we technically have long form and have eliminated the duplicated `price` variable; all prices are now in the `value` column. Notice that properties can now appear more than once in the table; conceptually, we have a row per sale rather than a row per property. 

We can tell `melt()` what to call the `variable` and `value` columns:

In [ ]:
sales_tidy = sales_wide1.melt(id_vars=['property_id','bedrooms'], 
                 var_name='sale_number',
                 value_name='price')

We still have empty extra rows where there was no sale in the original table, i.e. rows 3 and 5. We could use `dropna()` to get rid of these. A more generic approach would be to use filtering:

In [ ]:
sales = sales_wide1.melt(id_vars=['property_id','bedrooms'], 
                                     var_name='sale_number',
                                     value_name='price')
sales = sales[~sales['price'].isnull()]
sales

This is pretty good! Now we could run commands like `sales["price"].mean()` and get a sensible answer. We'll also be able to use the data easily to produce plots.

If you want a challenge, think about how you could convert `sales_wide2` to tidy form - it's a fair bit harder.

The inverse operation to `.melt()` is `.pivot()`:

In [ ]:
sales.pivot(index='property_id', columns='sale_number', values='price')

Let's have a look at our two remaining LTEE datasets. 

These two files record measurements of:

- the cell size for every population for the first 10,000 generations, measured every 1000 generations
- the relative fitness for every population for the first 10,000 generations, as measured by the growth rate of the strain relative to a reference strain.

In [ ]:
# These are TSV files, so specify tab as the delimiter
cellsize = pd.read_csv('ltee_cell_size.tsv', sep="\t")
fitness = pd.read_csv('ltee_relative_fitness.tsv', sep="\t")

In [ ]:
cellsize

To make life easier down the track, we'll rename the columns to match the variable and population names used in the `sampleruns` and `mutations` tables.

In [ ]:
cellsize.columns = (['Generation'] + ['Ara-{}'.format(n) for n  in range(1,7)] + 
                    ['Ara+{}'.format(n) for n  in range(1,7)])
cellsize.head()

In [ ]:
fitness.columns = (['Generation'] + ['Ara-{}'.format(n) for n  in range(1,7)] + 
                    ['Ara+{}'.format(n) for n  in range(1,7)])

#### Exercise

The cell size and relative fitness data is in wide form. Convert each one to tidy form. When thinking about which columns are "wide", it may help to aim to match the variables in the `sampleruns` and `mutations` tables.

## Merging data 

We can merge two datasets together by matching corresponding variables.

Our main options are `DataFrame.join()` and `pandas.merge()`. `merge()` is a little more flexible, so we'll demonstrate that.

Recall the `cars` dataset:

In [ ]:
cars.head()

The `origin` column here, which is a number 1-3, is actually intended to represent the country of origin. It's encoded as:

- USA : 1
- Europe : 2
- Japan : 3

Let's make a DataFrame to represent this mapping. We'll add a fourth code for Australia, which doesn't appear in the dataset!

In [ ]:
origin_codes = pd.DataFrame(
    {
        'origin': [1,2,3],
        'origin_country': ['USA','Europe','Japan']
    }
)
origin_codes

We can use `pandas.merge()` to join our `cars` table to our `origin_codes` table using the shared `origin` field, like so:

In [ ]:
cars_withorigin = pd.merge(cars, origin_codes)
cars_withorigin.sample(10)

This "just worked" because Pandas correctly deduced that the identically-named field(s) were the ones to match on. Sometimes we might need to be more verbose. In this case, this accomplishes the same thing as the above:

In [ ]:
# use origin from the left dataframe (cars) and from the right (origin_codes)
# use how="left" (keep all origin values that exist in the left dataframe)
cars_withorigin = pd.merge(cars, origin_codes, left_on='origin', right_on='origin', how="left")

Let's try to merge our `sampleinfo` and `mutations` columns. This time there are three shared fields: 'Strain ID', 'Population', and 'Generation'. In fact only 'Strain ID' is needed to uniquely identify  rows, but we want to specify all matching variables so that Pandas knows to only include each of these variables once in the resulting dataframe.

In [ ]:
# may need sampleruns.index.name = mutations.index.name = 'Index'
ltee = pd.merge(sampleruns, mutations, on=['Strain ID','Population','Generation'])
ltee.columns

In [ ]:
ltee.head()

Now that the metadata on Mutator phenotypes is together with the information on actual mutations, we could try exploring the relationships between these fields. Here are a couple of previews of ways to do this!

In [ ]:
sns.boxplot(data=ltee, x='Mutator', y='Total Mutations')

In [ ]:
ltee.groupby('Mutator')['Total Mutations'].mean()

#### Exercise

Merge the cell size data on to the `ltee` table with `pd.merge()`, using the Generation and Population variables. This is only possible if it's been converted to tidy form first, with variable names and values corresponding to those in `1tee`! You will probably want to set `how="left"` (if the cell size data is on the right), since the `ltee` table contains generations that don't exist in the cell size data, and we don't want to throw these rows away.

Do the same with the relative fitness data.

Have a look at missing and repeated values in the newly-added columns. What do you see?

## Writing to a file

To finish off, let's write out our tidied and merged table to a new file, for future analyses. To write to CSV, we can use the `to_csv` method.

In [ ]:
# Don't keep the index as we still have the Strain ID column
ltee.to_csv('ltee_solution.csv', index=False)